In [2]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt
import torch
import torch.nn as nn
import torch.optim as optim
import pickle

# 1. Excel 파일에서 댓글 불러오기
def load_comments_from_excel(files):
    all_comments = []
    for file in files:
        df = pd.read_excel(file)
        all_comments.extend(df['댓글 내용'].tolist())
    return all_comments

# 2. 한글만 필터링하는 함수
def filter_non_korean_comments(comments):
    korean_comments = []
    for comment in comments:
        if isinstance(comment, str) and re.fullmatch(r'[가-힣\s]+', comment):  # 한글과 공백만 포함된 경우
            korean_comments.append(comment)
    return korean_comments

# 3. TF-IDF 벡터화 (konlpy 형태소 분석기 사용)
okt = Okt()

def korean_tokenizer(text):
    return okt.morphs(text)

# 4. PyTorch 모델 정의
class CommentClassifier(nn.Module):
    def __init__(self, input_size):
        super(CommentClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 2)  # 2개의 클래스 (오버워치 관련 댓글 vs 비관련 댓글)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 5. 모델 훈련 및 평가
def train_model(X_train, y_train, input_size, device):
    model = CommentClassifier(input_size).to(device)  # 모델을 CUDA로 이동
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    X_train_tensor = torch.FloatTensor(X_train.toarray()).to(device)  # 데이터를 CUDA로 이동
    y_train_tensor = torch.LongTensor(y_train.values).to(device)  # 레이블도 CUDA로 이동

    epochs = 100
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")
    
    return model

# 6. 모델 테스트
def evaluate_model(model, X_test, y_test, device):
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test.toarray()).to(device)  # 데이터를 CUDA로 이동
        y_test_tensor = torch.LongTensor(y_test.values).to(device)  # 레이블도 CUDA로 이동
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs.data, 1)
        accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)
        print(f"모델 정확도: {accuracy:.4f}")

# 7. 모델 및 벡터라이저 저장
def save_model_and_vectorizer(model, vectorizer, model_path, vectorizer_path):
    torch.save(model.state_dict(), model_path)  # 모델 저장
    with open(vectorizer_path, 'wb') as f:  # 벡터라이저 저장
        pickle.dump(vectorizer, f)

# 8. 모델 및 벡터라이저 불러오기
def load_model_and_vectorizer(model_path, vectorizer_path, input_size, device):
    model = CommentClassifier(input_size).to(device)
    model.load_state_dict(torch.load(model_path))  # 저장된 모델 불러오기
    model.eval()
    
    with open(vectorizer_path, 'rb') as f:  # 저장된 벡터라이저 불러오기
        vectorizer = pickle.load(f)
    
    return model, vectorizer

# 9. 전체 실행
if __name__ == "__main__":
    # 1. Excel 파일 목록
    files = [f'result{i}.xlsx' for i in range(21)]  # 'result.xlsx' ~ 'result20.xlsx'
    
    # 2. Excel 파일에서 댓글 불러오기
    comments = load_comments_from_excel(files)
    
    # 3. 한글 댓글만 필터링
    korean_comments = filter_non_korean_comments(comments)
    
    # 4. 레이블 생성 (간단한 레이블링: 오버워치 관련 = 1, 비관련 = 0)
    df = pd.DataFrame(korean_comments, columns=['댓글 내용'])
    df['label'] = df['댓글 내용'].apply(lambda x: 1 if '오버워치' in x else 0)
    
    # 5. TF-IDF 벡터화
    vectorizer = TfidfVectorizer(tokenizer=korean_tokenizer, max_features=5000)
    X = vectorizer.fit_transform(df['댓글 내용'])
    y = df['label']
    
    # 6. 훈련 데이터와 테스트 데이터로 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 7. 장치 설정 (CUDA 사용)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 8. 모델 훈련
    model = train_model(X_train, y_train, input_size=X_train.shape[1], device=device)
    
    # 9. 모델 평가
    evaluate_model(model, X_test, y_test, device=device)
    
    # 10. 모델 및 벡터라이저 저장
    save_model_and_vectorizer(model, vectorizer, 'overwatch_model.pth', 'tfidf_vectorizer.pkl')
    
    # (옵션) 나중에 모델 및 벡터라이저 불러오기
    # model, vectorizer = load_model_and_vectorizer('overwatch_model.pth', 'tfidf_vectorizer.pkl', input_size=X_train.shape[1], device=device)


c:\Users\KDP-48\.conda\envs\TORCH_TEXT_38\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Epoch [10/100], Loss: 0.7008
Epoch [20/100], Loss: 0.6451
Epoch [30/100], Loss: 0.5763
Epoch [40/100], Loss: 0.4979
Epoch [50/100], Loss: 0.4168
Epoch [60/100], Loss: 0.3405
Epoch [70/100], Loss: 0.2747
Epoch [80/100], Loss: 0.2213
Epoch [90/100], Loss: 0.1792
Epoch [100/100], Loss: 0.1461
모델 정확도: 0.9638
